# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. We have been given 30 movies, 10 in each language, along with their reviews and synopses in separate CSV files named `movie_reviews_eng.csv`, `movie_reviews_fr.csv`, and `movie_reviews_sp.csv`.

- The first step of this project is to convert the French and Spanish reviews and synopses into English. This will allow us to analyze the sentiment of all reviews in the same language. We will be using pre-trained transformers from HuggingFace to achieve this task.

- Once the translations are complete, we will create a single dataframe that contains all the movies along with their reviews, synopses, and year of release in all three languages. This dataframe will be used to perform sentiment analysis on the reviews of each movie.

- Finally, we will use pretrained transformers from HuggingFace to analyze the sentiment of each review. The sentiment analysis results will be added to the dataframe. The final dataframe will have 30 rows


The output of the project will be a CSV file with a header row that includes column names such as **Title**, **Year**, **Synopsis**, **Review**, **Review Sentiment**, and **Original Language**. The **Original Language** column will indicate the language of the review and synopsis (*en/fr/sp*) before translation. The dataframe will consist of 30 rows, with each row corresponding to a movie.

In [62]:
import warnings
warnings.filterwarnings('ignore')

In [63]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline

### Get data from `.csv` files and then preprocess data

In [64]:
# use the `pd.read_csv()` function to read the movie_review_*.csv files into 3 separate pandas dataframes

# Note: All the dataframes would have different column names. For testing purposes
# you should have the following column names/headers -> [Title, Year, Synopsis, Review]

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    df_eng = pd.read_csv("data/movie_reviews_eng.csv")
    #TODO 1: Add your code here
    df_eng.rename(columns={'Movie / TV Series':'Title'}, inplace=True)
    df_eng['Original Language'] = 'ENGLISH'
    
    df_fr = pd.read_csv("data/movie_reviews_fr.csv")
    df_fr.rename(columns={"Titre": "Title", "Année": "Year", "Synopsis": "Synopsis", "Critiques": "Review"}, inplace=True)
    df_fr['Original Language'] = 'FRENCH'
    
    df_sp = pd.read_csv("data/movie_reviews_sp.csv")
    df_sp.rename(columns={"Título": "Title", "Año": "Year", "Sinopsis": "Synopsis", "Críticas": "Review"}, inplace=True)
    df_sp['Original Language'] = 'SPANISH'
    
    df = pd.concat([df_eng, df_fr, df_sp])
    df.reset_index(drop=True, inplace=True)
    
    return df

df = preprocess_data()

In [65]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
19,Babylon A.D.,2008,"Dans un futur lointain, un mercenaire doit esc...","""Ce film est un gâchis complet. Les personnage...",FRENCH
18,Les Visiteurs en Amérique,2000,Dans cette suite de la comédie française Les V...,"""Le film est une perte de temps totale. Les bl...",FRENCH
26,Toc Toc,2017,"En esta comedia española, un grupo de personas...","""Toc Toc es una película aburrida y poco origi...",SPANISH
28,Torrente: El brazo tonto de la ley,1998,"En esta comedia española, un policía corrupto ...","""Torrente es una película vulgar y ofensiva qu...",SPANISH
29,El Incidente,2014,"En esta película de terror mexicana, un grupo ...","""El Incidente es una película aburrida y sin s...",SPANISH
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",ENGLISH
25,Águila Roja,(2009-2016),Esta serie de televisión española sigue las av...,"""Águila Roja es una serie aburrida y poco inte...",SPANISH
12,Amélie,2001,Cette comédie romantique raconte l'histoire d'...,"""Amélie est un film absolument charmant qui vo...",FRENCH
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",ENGLISH
17,Astérix aux Jeux Olympiques,2008,Dans cette adaptation cinématographique de la ...,"""Ce film est une déception totale. Les blagues...",FRENCH


### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [66]:
# load translation models and tokenizers
# TODO 2: Update the code below
fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"
fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)
fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

# TODO 3: Complete the function below
def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors="pt")

    # generate the translation using the model
    outputs = model.generate(**inputs)

    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    return decoded

In [67]:
# TODO 4: Update the code below

# filter reviews in French and translate to English
fr_reviews = df[df['Original Language'] == 'FRENCH'].loc[:,'Review']
fr_reviews_en = []
for rev in fr_reviews:
    fr_reviews_en.append(translate(rev, fr_en_model, fr_en_tokenizer))

fr_reviews_en = pd.DataFrame(fr_reviews_en, columns=['Review'])

# filter synopsis in French and translate to English
fr_synopsis = df[df['Original Language'] == 'FRENCH'].loc[:,'Synopsis']
fr_synopsis_en = []
for syn in fr_synopsis:
    fr_synopsis_en.append(translate(syn, fr_en_model, fr_en_tokenizer))

fr_synopsis_en = pd.DataFrame(fr_synopsis_en, columns=['Synopsis'])

# filter reviews in Spanish and translate to English
es_reviews = df[df['Original Language'] == 'SPANISH'].loc[:,'Review']
es_reviews_en = []
for rev in es_reviews:
    es_reviews_en.append(translate(rev, es_en_model, es_en_tokenizer))

es_reviews_en = pd.DataFrame(es_reviews_en, columns=['Review'])

# filter synopsis in Spanish and translate to English
es_synopsis = df[df['Original Language'] == 'SPANISH'].loc[:,'Synopsis']
es_synopsis_en = []
for syn in es_synopsis:
    es_synopsis_en.append(translate(syn, es_en_model, es_en_tokenizer))

es_synopsis_en = pd.DataFrame(es_synopsis_en, columns=['Synopsis'])

# update dataframe with translated text
# add the translated reviews and synopsis - you can overwrite the existing data

en_reviews = pd.DataFrame(df[df['Original Language'] == 'ENGLISH'].loc[:,'Review'], columns=['Review'])
en_synopsis = pd.DataFrame(df[df['Original Language'] == 'ENGLISH'].loc[:,'Synopsis'], columns=['Synopsis'])

en_reviews_all = pd.concat([en_reviews, fr_reviews_en, es_reviews_en])
en_reviews_all.reset_index(drop=True, inplace=True)

en_synopsis_all = pd.concat([en_synopsis, fr_synopsis_en, es_synopsis_en])
en_synopsis_all.reset_index(drop=True, inplace=True)

df['Review'] = en_reviews_all['Review']
df['Synopsis'] = en_synopsis_all['Synopsis']

In [31]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",ENGLISH
18,Les Visiteurs en Amérique,2000,In this continuation of the French comedy The ...,"""The film is a total waste of time. The jokes ...",FRENCH
2,Forrest Gump,1994,Forrest Gump (Tom Hanks) is a simple man with ...,"""Forrest Gump is a heartwarming and inspiratio...",ENGLISH
12,Amélie,2001,This romantic comedy tells the story of Amélie...,"""Amélie is an absolutely charming film that wi...",FRENCH
10,La La Land,2016,This musical tells the story of a budding actr...,"""The Land is an absolutely beautiful film with...",FRENCH
13,Les Choristes,2004,This film tells the story of a music teacher w...,"""The Choristes are a beautiful film that will ...",FRENCH
17,Astérix aux Jeux Olympiques,2008,In this film adaptation of the popular comic s...,"""This film is a complete disappointment. The j...",FRENCH
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",ENGLISH
25,Águila Roja,(2009-2016),This Spanish television series follows the adv...,"""Red Eagle is a boring and uninteresting serie...",SPANISH
29,El Incidente,2014,"In this Mexican horror film, a group of people...","""The Incident is a boring and frightless film ...",SPANISH


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [35]:
# TODO 5: Update the code below
# load sentiment analysis model
model_name = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
sentiment_classifier = pipeline("sentiment-analysis", model=model_name)

# TODO 6: Complete the function below
def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    
    sentiments = sentiment_classifier(reviews)
    
    return sentiments

In [36]:
# TODO 7: Add code below for sentiment analysis
# perform sentiment analysis on reviews and store results in new column

reviews = df['Review'].tolist()
sentiments = analyze_sentiment(reviews, sentiment_classifier)
sentiments_df = pd.DataFrame.from_dict(sentiments)
sentiments_df.rename(columns={'label':'Review Sentiment'}, inplace=True)
sentiments_df.drop(['score'], axis=1, inplace=True)
df = pd.concat([df, sentiments_df], axis=1)
df.reset_index(drop=True, inplace=True)
df = df[['Title', 'Year', 'Synopsis', 'Review', 'Review Sentiment', 'Original Language']]

In [ ]:
df.sample(10)

In [61]:
# export the results to a .csv file
df.to_csv("result/reviews_with_sentiment.csv", index=False)